In [283]:
from mip import *



In [284]:
instance00 = {
    "m" : 1,
    "n" : 2,
    "l" : [15],
    "s" : [3, 2],
    "distances" : [
        [0, 4, 3,],
        [3, 0, 4,],
        [3, 4, 0,],
    ]
}

In [285]:
instance1 = {
    "m" : 3,
    "n" : 7,
    "l" : [15, 10, 7],
    "s" : [3, 2, 6, 8, 5, 4, 4],
    "distances" : [
        [0, 3, 3, 6, 5, 6, 6, 2],
        [3, 0, 4, 3, 4, 7, 7, 3],
        [3, 4, 0, 7, 6, 3, 5, 3],
        [6, 3, 7, 0, 3, 6, 6, 4],
        [5, 4, 6, 3, 0, 3, 3, 3],
        [6, 7, 3, 6, 3, 0, 2, 4],
        [6, 7, 5, 6, 3, 2, 0, 4],
        [2, 3, 3, 4, 3, 4, 4, 0]
    ]
}

In [286]:
"""instanceBig1 = {
    m = 6;
n = 9;
l = [190, 185, 185, 190, 195, 185];
s = [11, 11, 23, 16, 2, 1, 24, 14, 20];
distances = [0, 200, 119, 28, 180, 77, 145, 61, 123, 87, 
 		     200, 0, 81, 207, 38, 122, 55, 138, 76, 113, 
 		     119, 81, 0, 126, 69, 121, 26, 117, 91, 32, 
 		     28, 207, 126, 0, 187, 84, 152, 68, 130, 94, 
 		     170, 38, 79, 177, 0, 92, 58, 108, 46, 98, 
 		     77, 122, 121, 84, 102, 0, 100, 16, 46, 96, 
 		     145, 55, 26, 152, 58, 100, 0, 91, 70, 58, 
 		     61, 138, 113, 68, 118, 16, 91, 0, 62, 87, 
 		     123, 76, 91, 130, 56, 46, 70, 62, 0, 66, 
 		     87, 113, 32, 94, 94, 96, 58, 87, 66, 0 ];
}"""

instanceBig1 = {
    "m" : 6,
    "n" : 9,
    "l" : [190, 185, 185, 190, 195, 185],
    "s" : [11, 11, 23, 16, 2, 1, 24, 14, 20],
    "distances" : [
        [0, 200, 119, 28, 180, 77, 145, 61, 123, 87],
        [200, 0, 81, 207, 38, 122, 55, 138, 76, 113],
        [119, 81, 0, 126, 69, 121, 26, 117, 91, 32],
        [28, 207, 126, 0, 187, 84, 152, 68, 130, 94],
        [170, 38, 79, 177, 0, 92, 58, 108, 46, 98],
        [77, 122, 121, 84, 102, 0, 100, 16, 46, 96],
        [145, 55, 26, 152, 58, 100, 0, 91, 70, 58],
        [61, 138, 113, 68, 118, 16, 91, 0, 62, 87],
        [123, 76, 91, 130, 56, 46, 70, 62, 0, 66],
        [87, 113, 32, 94, 94, 96, 58, 87, 66, 0]
    ]
}

In [287]:
instance = instanceBig1

In [288]:
n = instance["n"]
m = instance["m"]
s = instance["s"]
l = np.array(instance["l"])
max_distance = np.max(instance["distances"], axis=1).sum()

# deposit = n + 1

# We have n + 1 nodes in total

model = Model()

# x[i, j1, j2] is 1 if the courier i travels from node j1 to node j2
x = model.add_var_tensor((m, n + 1, n + 1), var_type=BINARY, name="x")

# y[i] is the distance traveled by the courier i
y = model.add_var_tensor((m,), var_type=INTEGER, lb=0, ub=max_distance, name="y")

# t[i, j] is 1 if the courier i carries the package j
t = model.add_var_tensor((m, n), var_type=BINARY, name="t")

# o[i] is 1 if if the courier goes outside
#o = model.add_var_tensor((m,), var_type=BINARY, name="o")

# b[i, j] is 1 if you the courier i follows a path (starting from the deposit) that goes through the node j
#b = model.add_var_tensor((m, n + 1), var_type=BINARY, name="b")

# c[i, j1, j2] is 1 iff the courier i travels from node j1 to node j2 and b[i, j1] is true
#c = model.add_var_tensor((m, n + 1, n + 1), var_type=BINARY, name="c")

# z assigns an increasing number to each step of the courier
z = model.add_var_tensor((m, n + 1), var_type=INTEGER, lb=0, name="z")


"""# All couriers start at the starting deposit
for i in range(m):
    model += np.sum(x[i, n + 1, :]) == 1
    model += np.sum(x[i, :, n + 1]) == 0

# All couriers end at the arrival deposit
for i in range(m):
    model += np.sum(x[i, :, n + 2]) == 1
    model += np.sum(x[i, n + 2, :]) == 0"""

"""# If the courier stays home, he will not go anywhere
for i in range(m):
    # Blocks travels between packages
    model += x[i, :n, :n] <= o[i]
    # Blocks travels from deposit to packages
    # TODO: Check if it should be
    # model += np.sum(x[i, n, :n]) == 1
    # same for the other one
    model += x[i, n, :n] <= o[i]
    # (Opt.) Blocks travels from packages to deposit
    model += x[i, :n, n] <= o[i]

# If the courier goes outside, it cannot do deposit->deposit
for i in range(m):
    model += x[i, n, n] <= 1 - o[i]"""

# b is trivially true for the deposit
# model += b[:, n] == 1

"""a + b <= 2c
a >= c
b >= c"""

#c[i, j1, j2] = x[i, j1, j2] AND b[i, j1]

#b[i, j2] == sum(c[i, j1, j2] for j1 in range(n + 1)])


# If b[i, j1] is true and you are going from j1 to j2, then b[i, j2] is true
"""for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n + 1):
            # model += b[i, j1] + x[i, j1, j2] <= 2 * b[i, j2]
            # model += b[i, j2] <= b[i, j1]
            # model += b[i, j2] <= x[i, j1, j2]

            # model += b[i, j1] + x[i, j1, j2] - b[i, j2] <= 1

            model += c[i, j1, j2] <= b[i, j1]
            model += c[i, j1, j2] <= x[i, j1, j2]
            model += c[i, j1, j2] >= b[i, j1] + x[i, j1, j2] - 1

for i in range(m):
    for j in range(n + 1):
        model += np.sum(c[i, :, j]) == b[i, j]

# The courier i cannot pass through node j if b[i, j] is false
for i in range(m):
    for j in range(n + 1):
        # TODO: Optionally sum over j2
        model += x[i, j, :] <= b[i, j]
        model += x[i, :, j] <= b[i, j]"""

# z is 0 at the deposit
model += z[:, n] == 0

for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n):
            model += z[i, j2] == z[i, j1] + x[i, j1, j2]

for i in range(m):
    model += np.sum(x[i, n, :]) == 1
    model += np.sum(x[i, :, n]) == 1


# t[i, j] is 1 if the courier i carries the package j
for i in range(m):
    for j in range(n):
        model += np.sum(x[i, j, :]) == t[i, j]

# All packages are carried by exactly one courier
for j in range(n):
    model += np.sum(t[:, j]) == 1

# If a courier arrives to a package, it has to leave from that package
for i in range(m):
    for j in range(n):
        courier_arrives = np.sum(x[i, :, j])
        courier_leaves = np.sum(x[i, j, :])
        # (not arrives) or leaves
        model += (1 - courier_arrives) + courier_leaves >= 1


"""# Each node must be a middle stop exactly once
for j in range(n):
    model += np.sum(x[i, j, :]) == 1
    model += np.sum(x[i, :, j]) == 1"""

# Each courier does not exceed its capacity
for i in range(m):
    model += (t[i, :] @ s) <= l[i]

# y[i] is the distance traveled by the courier i
for i in range(m):
    model +=  np.sum(x[i, :, :] * np.array(instance["distances"])) == y[i]

# You cannot travel from a node to itself
for i in range(m):
    for j in range(n):
        model += x[i, j, j] == 0


z = model.add_var(name="z", var_type=INTEGER, lb=0, ub=max_distance)

for i in range(m):
    model += z >= y[i]

model.objective = minimize(z)

model.optimize()

<OptimizationStatus.INFEASIBLE: 1>

In [289]:
for i in range(m):
    for j1 in range(n + 1):
        for j2 in range(n + 1):
            if x[i, j1, j2].x == 1:
                print(f"Courier {i} travels from {j1} to {j2}")

In [290]:
for i in range(m):
    for j in range(n + 1):
        if b[i, j].x == 1:
            print(f"Courier {i} goes through {j}")

Courier 0 goes through 2
Courier 0 goes through 5
Courier 0 goes through 7
Courier 0 goes through 9
Courier 1 goes through 1
Courier 1 goes through 6
Courier 1 goes through 9
Courier 2 goes through 4
Courier 2 goes through 8
Courier 2 goes through 9
Courier 3 goes through 9
Courier 4 goes through 9
Courier 5 goes through 0
Courier 5 goes through 3
Courier 5 goes through 9


In [291]:
def find_next(i, node):
    for j in range(n + 1):
        if x[i, node, j].x == 1:
            return j


for i in range(m):
    steps = [n]
    current_node = n
    
    current_node = find_next(i, current_node)
    steps.append(current_node)
    while current_node != n:
        current_node = find_next(i, current_node)
        steps.append(current_node)
    
    print(f'Courier {i}:', ', '.join([str(node) for node in steps]))


AttributeError: 'LinExprTensor' object has no attribute 'x'

In [ ]:
print([y[i].x for i in range(m)])

[64.0, 102.0, 110.0, 0.0, 56.0, 32.0]


In [ ]:
print(model.objective_value)

110.0


In [ ]:
print([(i, j) for j in range(n) for i in range(m) if t[i, j].x == 1])

[(4, 0), (2, 1), (0, 2), (4, 3), (1, 4), (5, 5), (2, 6), (5, 7), (1, 8)]
